In [ ]:
from exporter import ScholarExporter
import sys

#import locale

#print(locale.getlocale())
#if locale.getlocale()[0] is None:
#        locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
#print(locale.getlocale())

In [10]:
#locale -a

In [2]:
import requests
from bs4 import BeautifulSoup

main_url = "http://www.chapter-living.com/properties/highbury"
re = requests.get(main_url)
soup = BeautifulSoup(re.text, "html.parser")
city_tags = soup.find_all('h2', class_="title")
cities_links = [main_url + tag.find('a').get('href','') if tag.find('a') else '' for tag in city_tags]
print( cities_links)

['http://www.chapter-living.com/properties/highbury/properties/highbury/rooms/2-bed-apartment', 'http://www.chapter-living.com/properties/highbury/properties/highbury/rooms/bronze-en-suite', 'http://www.chapter-living.com/properties/highbury/properties/highbury/rooms/silver-en-suite', 'http://www.chapter-living.com/properties/highbury/properties/highbury/rooms/bronze-studio', 'http://www.chapter-living.com/properties/highbury/properties/highbury/rooms/silver-studio', 'http://www.chapter-living.com/properties/highbury/properties/highbury/rooms/silver-premium-studio', 'http://www.chapter-living.com/properties/highbury/properties/highbury/rooms/gold-studio', 'http://www.chapter-living.com/properties/highbury/properties/highbury/rooms/gold-premium-studio', 'http://www.chapter-living.com/properties/highbury/properties/highbury/rooms/platinum-studio', 'http://www.chapter-living.com/properties/highbury/properties/highbury/rooms/diamond-studio', '', '', '', '']


In [7]:
s = ScholarExporter.from_user('tlMCbgwAAAAJ')  # Geoffrey Hinton user
t = s.export('index.html')

KeyError: 'data-href'